In [1]:
from pynetstim.coordinates import FreesurferCoords
from pynetstim.plotting import plotting_points 
from mayavi import mlab
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import measure
from scipy.io import loadmat
import nibabel as nib
import os

In [2]:
def create_surf_from_voi(voi_path, subject, subjects_dir, out_path, **kwargs):
    
    voi_data = nib.load(voi_path).get_fdata()
    verts, faces, norm, val = measure.marching_cubes(voi_data,**kwargs)
    
    fscoords = FreesurferCoords(verts, subject, subjects_dir, working_dir='.', coord_type='fsvoxel')
    nib.freesurfer.io.write_geometry(out_path, fscoords.coordinates['ras_coord'], faces)
    return fscoords.coordinates['ras_coord'], faces


def generate_diverging_colors(electrodes):
    """
    Generates a unique diverging color for each class.
    
    Parameters:
    num_classes (int): The number of different classes.
    
    Returns:
    list: A list of RGB tuples representing the colors.
    """
    # Use the seaborn diverging color palette
    palette = sns.color_palette("Spectral", len(electrodes))
    results = {elec:p for elec,p in zip(electrodes,palette)}
    return results


def extract_electrode(contact):
    try: 
        s = int(contact[-1])
        return extract_electrode(contact[0:-1])
    except:
        return contact

In [3]:
subject = 'freesurfer'
subjects_dir='/Users/ehsantadayon/Library/CloudStorage/Box-Box/ECLUB/SMV'
working_dir='.'

#coords 
coords_df = pd.read_csv(os.path.join(subjects_dir,'iceeg','clinical_elecs_all.txt'))
coords_df.head()

,name,x,y,z
0,AST1,-17.871750,18.889011,-33.093679
1,AST2,-25.063797,13.877090,-38.314198
2,AST3,-35.244767,11.303426,-36.611991
3,AST4,-44.541205,10.750185,-33.651610
4,AST5,-52.355438,10.785233,-27.187220


In [4]:
coords_df['elec'] = coords_df.name.apply(extract_electrode)

colors = generate_diverging_colors(np.unique(coords_df.elec))
coords_df['color'] = coords_df.elec.apply(lambda x: colors[x])


coords_df.head()

,name,x,y,z,elec,color
0,AST1,-17.871750,18.889011,-33.093679,AST,"(0.8662821991541715, 0.2901191849288735, 0.297..."
1,AST2,-25.063797,13.877090,-38.314198,AST,"(0.8662821991541715, 0.2901191849288735, 0.297..."
2,AST3,-35.244767,11.303426,-36.611991,AST,"(0.8662821991541715, 0.2901191849288735, 0.297..."
3,AST4,-44.541205,10.750185,-33.651610,AST,"(0.8662821991541715, 0.2901191849288735, 0.297..."
4,AST5,-52.355438,10.785233,-27.187220,AST,"(0.8662821991541715, 0.2901191849288735, 0.297..."


In [5]:
coords_xyz = coords_df[['x','y','z']].to_numpy()
fscoords = FreesurferCoords(coords_xyz, subject, subjects_dir, working_dir='.')
fscoords.add_trait('name',coords_df.name.to_list())
fscoords.add_trait('color',coords_df.color.to_list())

In [6]:
### surfs
### create lesion
lesion_surf = os.path.join(subjects_dir,'images','lesion_surf.nii.gz')
lesion_verts, lesion_faces = create_surf_from_voi(os.path.join(subjects_dir,'images','lesion_resampled.nii.gz'),subject,subjects_dir, lesion_surf)

In [11]:
p = plotting_points(fscoords,hemi='lh', show_skin=False,color=np.array([[1,0,0]]*fscoords.npoints),scale_factor=0.3,
                    show_name=True,name_shift='        ',alpha=0.3, cortex='low_contrast')


mlab.triangular_mesh(lesion_verts[:,0],lesion_verts[:,1],lesion_verts[:,2],lesion_faces,representation='mesh',color=(0.2,.8,0))

p.show()

2024-11-18 11:16:00.395 (49924.437s) [          5FF935]    vtkExtractEdges.cxx:427   INFO| Executing edge extractor: points are renumbered
2024-11-18 11:16:00.397 (49924.439s) [          5FF935]    vtkExtractEdges.cxx:543   INFO| Created 8454 edges
